# [TUTORIAL] Constrained Task Planning (Sweep)

This tutorial is written to instruct basic usage of the task & motion planning pipeline  
One Indy7 robot and several environment geometries will be added and floor-wiping task will be conducted.  
Here, the wiping task is defined as 1) contact with floor, 2) starting waypoint, 3) goal waypoint.  
Thus, any motion that satisfies the constraint will be generated; it may not look like real wiping.  

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,-0.3,0), (0,0,0)),
                INDY_IP),
    RobotConfig(1, RobotType.panda, ((0,0.3,0), (0,0,0)),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection_list
[False, False]


## create scene builder

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")
gscene = s_builder.create_gscene(crob)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## init planning pipeline

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [5]:
from pkg.geometry.geometry import *

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (0.3,0.5,0.01), (0.4,0,-0.005), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)
floor_col = gscene.create_safe(GEOTYPE.BOX, "floor_col", "base_link", (0.3,0.5,0.02), (0.4,0,-0.011), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=False, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

Please create a subscriber to the marker
   Use a production WSGI server instead.
 * Debug mode: off


In [6]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

## add wp

In [7]:
wp11 = gscene.create_safe(GEOTYPE.BOX, "wp11", "base_link", (0.08,0.08,0.01), (0.5,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp12 = gscene.create_safe(GEOTYPE.BOX, "wp12", "base_link", (0.08,0.08,0.01), (0.5,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line1 = gscene.create_safe(GEOTYPE.BOX, "wline1", "base_link", (0.01,0.5,1e-6), (0.5,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

wp21 = gscene.create_safe(GEOTYPE.BOX, "wp21", "base_link", (0.08,0.08,0.01), (0.4,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp22 = gscene.create_safe(GEOTYPE.BOX, "wp22", "base_link", (0.08,0.08,0.01), (0.4,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line2 = gscene.create_safe(GEOTYPE.BOX, "wline2", "base_link", (0.01,0.5,1e-6), (0.4,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

wp31 = gscene.create_safe(GEOTYPE.BOX, "wp31", "base_link", (0.08,0.08,0.01), (0.3,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp32 = gscene.create_safe(GEOTYPE.BOX, "wp32", "base_link", (0.08,0.08,0.01), (0.3,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line3 = gscene.create_safe(GEOTYPE.BOX, "wline3", "base_link", (0.01,0.5,1e-6), (0.3,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

## add brush

In [8]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="indy0_tcp", dims=(0.1,0.07,0.02), 
                   center=(0.0,0.0,0.14), rpy=(0,np.pi,0), color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=True)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="indy0_tcp", dims=(0.1,0.03,0.05), center=(0,0,0.035), rpy=(0,0,0), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=True, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face_col", link_name="indy0_tcp", dims=(0.1,0.06,0.028), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=True, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="indy0_tcp", dims=(0.1,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=True, parent="brush_body")

## add box

In [9]:
gbox1 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box1", link_name="base_link", dims=(0.05,0.05,0.05), 
                   center=(0.5,0.1,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

gbox2 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box2", link_name="base_link", dims=(0.05,0.05,0.05), 
                   center=(0.4,-0.1,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

# gbox3 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box3", link_name="base_link", dims=(0.05,0.05,0.05), 
#                    center=(0.3,-0.2,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

## Register binders

In [10]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool, SweepFramer, FixtureSlot

In [11]:
# gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
#                  dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [12]:
# pscene.create_binder(bname="grip0", gname="grip0", rname="indy0", _type=FixtureSlot, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="grip1", gname="grip1", rname="panda1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="brush_face", gname="brush_face", rname="indy0", _type=SweepFramer, point=(0,0,-0.015), rpy=(0,0,0))

## add objects

In [13]:
from pkg.planning.constraint.constraint_subject import \
        CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepFrame, SweepTask, BoxObject, FixturePoint, AbstractObject

In [14]:
brush_handle = gscene.NAME_DICT["brush_handle"]
brush_face = gscene.NAME_DICT["brush_face"]
# brush = pscene.create_subject(oname="brush", gname="brush_body", _type=CustomObject, 
#                              action_points_dict = {"handle": FixturePoint("handle", brush_handle, [0,0,0], [np.pi/2,0,0]),
#                                                    "face": PlacePoint("face", brush_face, [0,0,-0.015], [0,0,0])})

In [15]:
box1 = pscene.create_subject(oname="box1", gname="box1", _type=BoxObject, hexahedral=True)
box2 = pscene.create_subject(oname="box2", gname="box2", _type=BoxObject, hexahedral=True)
# box3= pscene.create_subject(oname="box3", gname="box3", _type=BoxObject, hexahedral=True)

In [16]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [17]:
sweep1 = pscene.create_subject(oname="sweep1", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp11": SweepFrame("wp11", wp11, [0,0,0.005], [0,0,0]),
                                                   "wp12": SweepFrame("wp12", wp12, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line1)
sweep2 = pscene.create_subject(oname="sweep2", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp21": SweepFrame("wp21", wp21, [0,0,0.005], [0,0,0]),
                                                   "wp22": SweepFrame("wp22", wp22, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line2)
sweep3 = pscene.create_subject(oname="sweep3", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp31": SweepFrame("wp31", wp31, [0,0,0.005], [0,0,0]),
                                                   "wp32": SweepFrame("wp32", wp32, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line3)

### planners

In [18]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion(mplan)
ppline.set_sampler(tplan)

## motion filters

In [19]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker

gcheck = GraspChecker(pscene, 
                      end_link_couple_dict= {
                          "indy0_tcp": ["indy0_tcp", "indy0_link6"],
                          "panda1_hand": ["panda1_hand", "panda1_link6"],
                          "base_link":["base_link"]})
rcheck = ReachChecker(pscene)
checkers_all = [gcheck, rcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [20]:
mplan.motion_filters = checkers_all

In [21]:
gscene.show_pose(crob.home_pose)

## Set initial condition

In [22]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.update_state(crob.home_pose)
print(initial_state.node)

('floor', 'floor', 0, 0, 0)


In [23]:
pscene.subject_name_list

['box1', 'box2', 'sweep1', 'sweep2', 'sweep3']

# Node Sampler

In [24]:
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler

In [25]:

##
# @class NodeSampler
# @biref uniformly sample nodes by decaying probability of sampled node
class GrowingSampler:
    def __init__(self, log2_decay=1):
        self.log2_decay = log2_decay

    def init(self, tplan, multiprocess_manager):
        self.node_dict = tplan.node_dict
        self.multiprocess_manager = multiprocess_manager
        if multiprocess_manager is not None:
            self.log2_prob_dict = multiprocess_manager.dict()
            self.prob_lock = multiprocess_manager.Lock()
        else:
            self.log2_prob_dict = dict()
            self.prob_lock = DummyBlock()

    def __call__(self, nodes):
        nodes = list(nodes)
        for node in nodes:
            if node not in self.log2_prob_dict:
                self.log2_prob_dict[node] = 1.0
        probs = [np.exp2(self.log2_prob_dict[node]) for node in nodes]
        sumprobs = np.sum(probs)
        if sumprobs == 0:
            print("probability saturated")
            probs = np.ones_like(probs)/len(probs)
        else:
            probs = np.divide(probs, sumprobs)
        i_node = np.random.choice(len(nodes), p=probs)
        node = nodes[i_node]
        with self.prob_lock:
            self.log2_prob_dict[node] = self.log2_prob_dict[node] - self.log2_decay
        return node

    
##
# @class NodeSampler
# @biref uniformly sample nodes by decaying probability of sampled node
class PenaltyNodeSampler:
    def __init__(self, log2_decay=1, log2_count_decay=1):
        self.log2_decay = log2_decay
        self.log2_count_decay = log2_count_decay

    def init(self, tplan, multiprocess_manager):
        self.node_dict = tplan.node_dict
        self.multiprocess_manager = multiprocess_manager
        if multiprocess_manager is not None:
            self.log2_prob_dict = multiprocess_manager.dict()
            self.log2_count_dict = multiprocess_manager.dict()
            self.prob_lock = multiprocess_manager.Lock()
        else:
            self.log2_prob_dict = dict()
            self.log2_count_dict = dict()
            self.prob_lock = DummyBlock()

        for k in self.node_dict.keys():
            self.log2_prob_dict[k] = 1.0
            self.log2_count_dict[k] = 0.0

    def __call__(self, nodes):
        nodes = list(nodes)
        probs = [np.exp2(self.log2_prob_dict[node]+self.log2_count_dict[node]) for node in nodes]
        sumprobs = np.sum(probs)
        if sumprobs == 0:
            print("probability saturated")
            probs = np.ones_like(probs)/len(probs)
        else:
            probs = np.divide(probs, sumprobs)
        i_node = np.random.choice(len(nodes), p=probs)
        node = nodes[i_node]
        with self.prob_lock:
            self.log2_prob_dict[node] = self.log2_prob_dict[node] - self.log2_decay
        return node
    
    def update(self, snode_src, snode_new, connection_result):
        if connection_result:
            self.log2_count_dict[snode_new.state.node] = self.log2_count_dict[snode_new.state.node] - self.log2_count_decay
            

In [26]:
tplan.new_node_sampler = PenaltyNodeSampler(3, 1)
tplan.parent_node_sampler = UniformNodeSampler(3)
# tplan.parent_snode_sampler = GrowingSampler(3)

In [27]:
class BindingSampler:
    def __init__(self, gamma_decay=0.8):
        self.gamma_decay = gamma_decay

    def init(self, tplan, multiprocess_manager):
        self.multiprocess_manager = multiprocess_manager
        if multiprocess_manager is not None:
            self.prob_dict = multiprocess_manager.dict()
            self.prob_lock = multiprocess_manager.Lock()
        else:
            self.prob_dict = dict()
            self.prob_lock = DummyBlock()

    def __call__(self, nodes):
        nodes = list(nodes)
        probs = [self.prob_dict[node] for node in nodes]
        probs = np.divide(probs, np.sum(probs))
        i_node = np.random.choice(len(nodes), p=probs)
        node = nodes[i_node]
        with self.prob_lock:
            self.prob_dict[node] = self.prob_dict[node] * self.gamma_decay
        return node

# CustomRule

In [28]:
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [29]:
class CustomRule:
    def __init__(self, pscene):
        self.pscene = pscene
        self.chain_dict = pscene.get_robot_chain_dict()
        
    def init(self, tplan, multiprocess_manager):
        self.multiprocess_manager = multiprocess_manager
        if multiprocess_manager is not None:
            self.call_count_dict = multiprocess_manager.dict()
            self.count_lock = multiprocess_manager.Lock()
        else:
            self.call_count_dict = dict()
            self.count_lock = DummyBlock()
            
        no_enter_sidxes = [stype==SweepLineTask for stype in enumerate(self.pscene.subject_type_list)]
        no_enter_initials = [tplan.initial_state.node[sidx] for sidx in no_enter_sidxes]
        self.sorted_unstop_keys = sorted(tplan.unstoppable_terminals.keys())
            
        self.node_dict={}
        self.enter_dict={}
        for node, leafs in tplan.node_dict.items():
            self.node_dict[node] = deepcopy(leafs)
            if not all([node[k] in terms for k, terms in tplan.unstoppable_terminals.items()]):
                tplan.node_dict[node] = set([]) ## unstoppable node change will be reserved by this custom rule
            else:
                # entering to unstoppable terminal is controlled by this rule
                self.enter_dict[node] = set([leaf for leaf in leafs 
                                             if any([leaf[k] not in terms for k, terms in tplan.unstoppable_terminals.items()])
                                             and all([leaf[k]>=leaf[k+1] for k in self.sorted_unstop_keys[:-1]])]) # the task is done in order
                tplan.node_dict[node] = set([leaf for leaf in leafs 
                                             if all([leaf[k] in terms for k, terms in tplan.unstoppable_terminals.items()])
                                            ])

        self.node_parent_dict={}
        for node, parents in tplan.node_parent_dict.items():
            self.node_parent_dict[node] = deepcopy(parents)
            tplan.node_parent_dict[node] = parents = set(
                [parent for parent in parents ## unstoppable node change will be reserved by this custom rule
                 if all([parent[k] in terms for k, terms in tplan.unstoppable_terminals.items()])])
        
    def __call__(self, tplan, snode_src, snode_new, connection_result):
        print("CustomRule call")
        stack_res = False
        stack_items = []
        if connection_result and snode_new.state.node in self.enter_dict:
            with tplan.snode_dict_lock:
                nb_nodes = list(tplan.neighbor_nodes.keys())
                for nb_node in nb_nodes:
                    for ukey in self.sorted_unstop_keys:
                        if nb_node[ukey] < snode_new.state.node[ukey]:
                            del tplan.neighbor_nodes[nb_node]
                            break # task is done in order
        
        if connection_result and snode_new.state.node in self.enter_dict:
            stack_res, stack_items = True, list(self.enter_dict[snode_new.state.node])
            
        if snode_src is not None:
            with self.count_lock:
                if snode_src.idx in self.call_count_dict:
                    self.call_count_dict[snode_src.idx] = self.call_count_dict[snode_src.idx] + [snode_new.state.node]
                else:
                    self.call_count_dict[snode_src.idx] = [snode_new.state.node]
            
            node_src = snode_src.state.node
            node_new = snode_new.state.node
            diff_sidxes = np.where([ ntem_s != ntem_g for ntem_s, ntem_g in zip(node_src, node_new)])[0]
            print("{}->{} , diff: {}".format(node_src, node_new, diff_sidxes))
            if len(diff_sidxes)==0:
                return stack_res, stack_items
            diff_sidx = diff_sidxes[0]
            diff_sname = self.pscene.subject_name_list[diff_sidx]
            diff_subject = self.pscene.subject_dict[diff_sname]
            
            if isinstance(diff_subject, SweepLineTask):
                print("Rule for SweepLineTask")
                with tplan.snode_dict_lock:
                    if node_src[diff_sidx] not in tplan.unstoppable_terminals[diff_sidx]: # from intermediate wp -> remove access to them
                        snode_list = tplan.node_snode_dict[node_src]
                        if snode_src.idx in snode_list:
                            print("Removing {} from {}: {} not in {}".format(snode_src.idx, snode_list, node_src[diff_sidx], tplan.unstoppable_terminals[diff_sidx]))
                            print(snode_list)
                            snode_list.remove(snode_src.idx)
                            tplan.node_snode_dict[node_src] = snode_list
                            print(tplan.node_snode_dict[node_src])
                        if len(snode_list) == 0 and node_new in tplan.neighbor_nodes:
                            del tplan.neighbor_nodes[node_new]
                    if connection_result:
                        if node_new[diff_sidx] in tplan.unstoppable_terminals[diff_sidx]: ## in terminal condition
                            print("Check home: {} in {}".format(node_new[diff_sidx], tplan.unstoppable_terminals[diff_sidx]))
                            link_name = self.pscene.gscene.NAME_DICT[snode_new.state.binding_state[diff_sidx][-1]].link_name
                            rname_candis = [rname for rname, chain_vals in self.chain_dict.items() if 
                                            link_name in chain_vals['link_names']]
                            if len(rname_candis)==0:
                                print("no robot candis")
                            else:
                                print("reserve homing = {}".format(snode_new.idx))
                                newstate = snode_new.state.copy(self.pscene)
                                jidxes = self.pscene.combined_robot.idx_dict[rname_candis[0]]
                                newstate.Q[jidxes] = self.pscene.combined_robot.home_pose[jidxes]
                                return True, [newstate] + stack_items
                        else:## not in terminal condition
                            print("Reserve next wp from {}".format(snode_new.idx))
                            snode_list = tplan.node_snode_dict[node_new]
                            if snode_new.idx in snode_list:
                                print("Removing {} from {} as next wp is reserved".format(snode_new.idx, snode_list))
                                snode_list.remove(snode_new.idx)
                                tplan.node_snode_dict[node_new] = snode_list
                            next_node_candis = list(self.node_dict[node_new])
                            assert len(next_node_candis) == 1, "non-terminal sweep task should have only 1 leaf ({}) {}-{}".format(diff_sidx, node_new, next_node_candis)
                            return True, next_node_candis + stack_items
                return stack_res, stack_items
            
            if isinstance(diff_subject, AbstractObject):
                print("Rule for AbstractObject")
                if connection_result:
                    link_name1 = snode_src.state.state_param[diff_sname][0]
                    link_name2 = snode_new.state.state_param[diff_sname][0]
                    rname_candis = [rname for rname, chain_vals in self.chain_dict.items() if 
                                   link_name1 in chain_vals['link_names'] or link_name2 in chain_vals['link_names']]
                    if len(rname_candis)==0:
                        print("no robot candis")
                        return stack_res, stack_items
                    else:
                        print("try homing")
                        newstate = snode_new.state.copy(self.pscene)
                        jidxes = self.pscene.combined_robot.idx_dict[rname_candis[0]]
                        newstate.Q[jidxes] = self.pscene.combined_robot.home_pose[jidxes]
                        return True, [newstate] + stack_items
        return stack_res, stack_items

In [30]:
tplan.custom_rule = CustomRule(pscene)

In [31]:
gtimer.reset()
goal_nodes = [("floor","floor", 2, 0, 0), ("grip1","floor", 2, 0, 0), ("floor","grip1", 2, 0, 0)]
# goal_nodes = [("floor","floor","floor", 2, 2, 2)]
gtimer.tic("plan")
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=100, multiprocess=True, timeout=1, timeout_constrained=5)
gtimer.toc("plan")
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.sort_schedule(schedules)
snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

Use 20/20 agents
CustomRule call
make custom reservation: [('floor', 'floor', 1, 0, 0)]
got reserved one from 0
try: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0)
not interpolating
sampled one from 0
try: 0 - ('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0)
sampled one from 0
not interpolating
try: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0)
not interpolating
CustomRule call
sampled one from 0
CustomRule call
('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) , diff: [0]
Rule for AbstractObject
('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) , diff: [1]
try: 0 - ('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0)
Rule for AbstractObject
result: 0 - ('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) = fail
not interpolating
sampled one from 0
result: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) = fail
try: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0)
not interpolating
transition motion trie

sampled one from 0
sampled one from 0
not interpolating
Rule for AbstractObject
not interpolating
not interpolating
try: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0)
result: 0 - ('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) = fail
CustomRule call
('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) , diff: [0]
not interpolating
CustomRule call
sampled one from 0
try: 0 - ('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0)
try: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0)
try: 0 - ('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0)
CustomRule call
Rule for AbstractObject
not interpolating
CustomRule call
not interpolating
('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) , diff: [1]
not interpolating
sampled one from 0
not interpolating
('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) , diff: [0]
not interpolating
('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) , diff: [0]
CustomRule call
CustomRule call
Cu

CustomRule call
try: 4 - ('floor', 'grip1', 0, 0, 0)->('floor', 'grip1', 0, 0, 0)
result: 3 - ('floor', 'grip1', 0, 0, 0)->('floor', 'floor', 0, 0, 0) = fail
got reserved one from 4
make custom reservation: [('floor', 'grip1', 1, 0, 0)]
try: 4 - ('floor', 'grip1', 0, 0, 0)->('floor', 'grip1', 1, 0, 0)
result: 2 - ('floor', 'grip1', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) = success
Rule for AbstractObject
make custom reservation: [<pkg.planning.scene.State instance at 0x7f00290d88c0>, ('floor', 'grip1', 1, 0, 0)]
('floor', 'grip1', 0, 0, 0)->('floor', 'floor', 0, 0, 0) , diff: [1]
result: 3 - ('floor', 'grip1', 0, 0, 0)->('floor', 'floor', 0, 0, 0) = success
try homing
branching: 3->6 (1.02/100.0 s, steps/err: 21(123.723983765 ms)/0.00116560951311)
result: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) = success
branching: 2->7 (1.02/100.0 s, steps/err: 32(91.2368297577 ms)/1.30922857482e-15)
('floor', 'grip1', 0, 0, 0)->('floor', 'floor', 0, 0, 0) , diff: [1]
not interpolat

got reserved one from 11
result: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) = success
('floor', 'grip1', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) , diff: []
try: 10 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0)
('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0) , diff: [2]
transition motion tried: False
got reserved one from 11
try: 11 - ('grip1', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0)
try: 11 - ('grip1', 'floor', 0, 0, 0)->('grip1', 'floor', 1, 0, 0)
make custom reservation: [('floor', 'grip1', 1, 0, 0)]
result: 5 - ('floor', 'grip1', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) = success
branching: 0->14 (1.3/100.0 s, steps/err: 35(801.419019699 ms)/0.00111141669299)
CustomRule call
not interpolating
got reserved one from 15
('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) , diff: [1]
Rule for SweepLineTask
Removing 18 from [18] as next wp is reserved
Reserve next wp from 18
not interpolating
try: 15 - ('floor', 'floor', 0, 0, 0)->('floor', 

got reserved one from 25
try: 23 - ('floor', 'grip1', 0, 0, 0)->('floor', 'grip1', 1, 0, 0)
('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) , diff: [0]
try: 25 - ('floor', 'grip1', 1, 0, 0)->('floor', 'grip1', 2, 0, 0)
Rule for SweepLineTask
interpolating
not interpolating
result: 22 - ('floor', 'grip1', 1, 0, 0)->('floor', 'grip1', 2, 0, 0) = fail
CustomRule call
Rule for AbstractObject
('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0) , diff: []
got reserved one from 27
make custom reservation: [('floor', 'floor', 1, 0, 0)]
('floor', 'floor', 1, 0, 0)->('floor', 'floor', 2, 0, 0) , diff: [2]
try: 27 - ('grip1', 'floor', 0, 0, 0)->('grip1', 'floor', 1, 0, 0)
got reserved one from 26
not interpolating
result: 13 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0) = success
try: 26 - ('floor', 'grip1', 1, 0, 0)->('floor', 'grip1', 2, 0, 0)
Rule for SweepLineTask
result: 21 - ('floor', 'floor', 1, 0, 0)->('floor', 'floor', 2, 0, 0) = fail
branching: 13->28 (1.76/10

...

not interpolating
try: 45 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0)
('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) , diff: [0]
try: 70 - ('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0)
result: 69 - ('grip1', 'floor', 0, 0, 0)->('grip1', 'floor', 1, 0, 0) = success
branching: 69->71 (4.22/100.0 s, steps/err: 12(103.211164474 ms)/0.00141456553512)
not interpolating
not interpolating
transition motion tried: True
Rule for AbstractObject
CustomRule call
got reserved one from 71
CustomRule call
CustomRule call
('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) , diff: [1]
result: 47 - ('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) = fail
('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0) , diff: [2]
try: 71 - ('grip1', 'floor', 1, 0, 0)->('grip1', 'floor', 2, 0, 0)
sampled one from 15
Rule for SweepLineTask
('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) , diff: [1]
Rule for AbstractObject
Rule for AbstractObject
result: 67 

got reserved one from 78
transition motion tried: True
not interpolating
('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0) , diff: [2]
CustomRule call
Rule for SweepLineTask
CustomRule call
try: 78 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0)
not interpolating
('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) , diff: [0]
got reserved one from 77
got reserved one from 77
try: 77 - ('grip1', 'floor', 0, 0, 0)->('grip1', 'floor', 1, 0, 0)
Rule for AbstractObject
joint motion tried: True
('floor', 'grip1', 0, 0, 0)->('floor', 'floor', 0, 0, 0) , diff: [1]
try: 77 - ('grip1', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0)
result: 64 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0) = fail
sampled one from 44
try: 44 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0)
Rule for AbstractObject
CustomRule call
('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0) , diff: []
not interpolating
result: 37 - ('floor', 'floor', 0, 0, 0)->('grip1', 'floo

try: 74 - ('floor', 'grip1', 0, 0, 0)->('floor', 'floor', 0, 0, 0)
try homing
not interpolating
make custom reservation: [<pkg.planning.scene.State instance at 0x7f00290ac0f0>, ('floor', 'grip1', 1, 0, 0)]
result: 39 - ('grip1', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0) = success
not interpolating
branching: 83->87 (5.51/100.0 s, steps/err: 12(107.952833176 ms)/0.00177999426702)
result: 35 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) = success
branching: 39->86 (5.52/100.0 s, steps/err: 22(545.469999313 ms)/0.00173480115325)
make custom reservation: [<pkg.planning.scene.State instance at 0x7f00290d8fa0>, ('grip1', 'floor', 1, 0, 0)]
branching: 35->89 (5.54/100.0 s, steps/err: 29(1378.74984741 ms)/0.00133097934953)
CustomRule call
CustomRule call
got reserved one from 87
try: 87 - ('grip1', 'floor', 1, 0, 0)->('grip1', 'floor', 2, 0, 0)
got reserved one from 86
transition motion tried: True
('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) , diff: [0]
result: 3

try: 23 - ('floor', 'grip1', 0, 0, 0)->('floor', 'floor', 0, 0, 0)
try: 24 - ('floor', 'grip1', 0, 0, 0)->('floor', 'floor', 0, 0, 0)
('floor', 'grip1', 0, 0, 0)->('floor', 'grip1', 1, 0, 0) , diff: [2]
constrained motion tried: True
sampled one from 10
not interpolating
CustomRule call
Rule for SweepLineTask
result: 91 - ('floor', 'grip1', 0, 0, 0)->('floor', 'grip1', 1, 0, 0) = fail
not interpolating
result: 66 - ('floor', 'floor', 1, 0, 0)->('floor', 'floor', 2, 0, 0) = success
try: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0)
('grip1', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0) , diff: [0]
not interpolating
not interpolating
branching: 66->94 (6.01/100.0 s, steps/err: 55(2104.34103012 ms)/0.00171131759282)
try: 10 - ('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0)
Rule for AbstractObject
sampled one from 94
result: 57 - ('grip1', 'floor', 1, 0, 0)->('grip1', 'floor', 2, 0, 0) = success
CustomRule call
branching: 57->95 (6.03/100.0 s, steps/err: 55(2508.6

branching: 91->99 (6.28/100.0 s, steps/err: 25(312.422990799 ms)/1.04359317909e-15)
======================= terminated 6: first answer acquired from other agent ===============================
======================= terminated 3: first answer acquired from other agent ===============================
transition motion tried: True
CustomRule call
('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) , diff: [1]
Rule for AbstractObject
try homing
make custom reservation: [<pkg.planning.scene.State instance at 0x7f00290bdc80>, ('floor', 'grip1', 1, 0, 0)]
result: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) = success
branching: 0->100 (6.34/100.0 s, steps/err: 26(315.618038177 ms)/0.0015458798251)
======================= terminated 18: first answer acquired from other agent ===============================
transition motion tried: False
CustomRule call
('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0) , diff: [2]
Rule for SweepLineTask
result: 86 - ('floor', 'floo

In [32]:
print(gtimer)

plan: 	6125.0 ms/1 = 6124.775 ms (6124.775/6124.775)



In [33]:
from_state = snode_schedule[-1].state
gtimer.reset()
goal_nodes = [("floor","floor", 2, 2, 0), ("grip1","floor", 2, 2, 0), ("floor","grip1", 2, 2, 0)]
# goal_nodes = [("floor","floor","floor", 2, 2, 2)]
gtimer.tic("plan")
ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
              timeout_loop=100, multiprocess=True, timeout=1, timeout_constrained=5,
              wait_proc=False)
gtimer.toc("plan")

Use 20/20 agents
CustomRule call
make custom reservation: [('grip1', 'floor', 2, 1, 0)]
got reserved one from 0
try: 0 - ('grip1', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 1, 0)
not interpolating
sampled one from 0
try: 0 - ('grip1', 'floor', 2, 0, 0)->('floor', 'floor', 2, 0, 0)
sampled one from 0
try: 0 - ('grip1', 'floor', 2, 0, 0)->('floor', 'floor', 2, 0, 0)
not interpolating
not interpolating
sampled one from 0
try: 0 - ('grip1', 'floor', 2, 0, 0)->('floor', 'floor', 2, 0, 0)
CustomRule call
('grip1', 'floor', 2, 0, 0)->('floor', 'floor', 2, 0, 0) , diff: [0]
not interpolating
Rule for AbstractObject
result: 0 - ('grip1', 'floor', 2, 0, 0)->('floor', 'floor', 2, 0, 0) = fail
CustomRule call
transition motion tried: True
sampled one from 0
try: 0 - ('grip1', 'floor', 2, 0, 0)->('floor', 'floor', 2, 0, 0)
('grip1', 'floor', 2, 0, 0)->('floor', 'floor', 2, 0, 0) , diff: [0]
Rule for AbstractObject
not interpolating
result: 0 - ('grip1', 'floor', 2, 0, 0)->('floor', 'floor', 2, 0, 0)

404.296875

result: 0 - ('grip1', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 1, 0) = success
branching: 0->1 (0.41/100.0 s, steps/err: 7(121.814966202 ms)/0.00156103495341)
got reserved one from 1
try: 1 - ('grip1', 'floor', 2, 1, 0)->('grip1', 'floor', 2, 2, 0)
interpolating
sampled one from 0
try: 0 - ('grip1', 'floor', 2, 0, 0)->('floor', 'floor', 2, 0, 0)
not interpolating
CustomRule call
('grip1', 'floor', 2, 1, 0)->('grip1', 'floor', 2, 2, 0) , diff: [3]
CustomRule call
Rule for SweepLineTask
('grip1', 'floor', 2, 0, 0)->('floor', 'floor', 2, 0, 0) , diff: [0]
Rule for AbstractObject
result: 0 - ('grip1', 'floor', 2, 0, 0)->('floor', 'floor', 2, 0, 0) = fail
result: 1 - ('grip1', 'floor', 2, 1, 0)->('grip1', 'floor', 2, 2, 0) = fail
sampled one from 0
try: 0 - ('grip1', 'floor', 2, 0, 0)->('floor', 'floor', 2, 0, 0)
transition motion tried: True
CustomRule call
not interpolating
sampled one from 0
CustomRule call
('grip1', 'floor', 2, 0, 0)->('floor', 'floor', 2, 0, 0) , diff: [0]
('grip1', 'f

('floor', 'floor', 2, 0, 0)->('floor', 'floor', 2, 1, 0) , diff: [3]
('floor', 'floor', 2, 0, 0)->('floor', 'grip1', 2, 0, 0) , diff: [1]
('floor', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 0, 0) , diff: [0]
Rule for SweepLineTask
sampled one from 2
('grip1', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 0, 0) , diff: []
make custom reservation: [('grip1', 'floor', 2, 1, 0)]
Rule for AbstractObject
try: 2 - ('floor', 'floor', 2, 0, 0)->('floor', 'grip1', 2, 0, 0)
result: 6 - ('floor', 'floor', 2, 0, 0)->('floor', 'floor', 2, 1, 0) = fail
got reserved one from 7
Rule for AbstractObject
result: 3 - ('grip1', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 0, 0) = success
result: 6 - ('floor', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 0, 0) = fail
result: 2 - ('floor', 'floor', 2, 0, 0)->('floor', 'grip1', 2, 0, 0) = fail
try: 7 - ('grip1', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 1, 0)
sampled one from 6
not interpolating
try: 6 - ('floor', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 0, 0)
sampled one fr

Rule for AbstractObject
not interpolating
result: 4 - ('floor', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 0, 0) = fail
CustomRule call
('floor', 'floor', 2, 0, 0)->('floor', 'grip1', 2, 0, 0) , diff: [1]
sampled one from 3
result: 8 - ('grip1', 'floor', 2, 1, 0)->('grip1', 'floor', 2, 2, 0) = fail
not interpolating
not interpolating
sampled one from 4
result: 6 - ('floor', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 0, 0) = fail
('grip1', 'floor', 2, 0, 0)->('floor', 'floor', 2, 0, 0) , diff: [0]
('grip1', 'floor', 2, 0, 0)->('floor', 'floor', 2, 0, 0) , diff: [0]
Rule for AbstractObject
try: 3 - ('grip1', 'floor', 2, 0, 0)->('floor', 'floor', 2, 0, 0)
CustomRule call
result: 4 - ('floor', 'floor', 2, 0, 0)->('floor', 'grip1', 2, 0, 0) = fail
try: 0 - ('grip1', 'floor', 2, 0, 0)->('floor', 'floor', 2, 0, 0)
sampled one from 0
('grip1', 'floor', 2, 0, 0)->('floor', 'floor', 2, 0, 0) , diff: [0]
try: 4 - ('floor', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 0, 0)
Rule for AbstractObject
sampled

...

not interpolating
branching: 69->76 (6.18/100.0 s, steps/err: 7(235.897064209 ms)/0.00133918047931)
CustomRule call
got reserved one from 75
Rule for SweepLineTask
result: 57 - ('floor', 'grip1', 2, 0, 0)->('floor', 'grip1', 2, 1, 0) = fail
got reserved one from 76
('floor', 'grip1', 2, 0, 0)->('floor', 'floor', 2, 0, 0) , diff: [1]
try: 75 - ('floor', 'floor', 2, 0, 0)->('floor', 'floor', 2, 1, 0)
result: 72 - ('floor', 'floor', 2, 1, 0)->('floor', 'floor', 2, 2, 0) = fail
sampled one from 73
sampled one from 68
sampled one from 33
Rule for AbstractObject
try: 76 - ('grip1', 'floor', 2, 1, 0)->('grip1', 'floor', 2, 2, 0)
transition motion tried: True
sampled one from 74
transition motion tried: True
interpolating
try: 68 - ('floor', 'floor', 2, 0, 0)->('floor', 'grip1', 2, 0, 0)
result: 74 - ('floor', 'grip1', 2, 0, 0)->('floor', 'floor', 2, 0, 0) = fail
try: 74 - ('floor', 'grip1', 2, 0, 0)->('floor', 'floor', 2, 0, 0)
sampled one from 75
CustomRule call
not interpolating
try: 73 - (

('floor', 'floor', 2, 1, 0)->('floor', 'floor', 2, 2, 0) , diff: [3]
try: 81 - ('floor', 'floor', 2, 1, 0)->('floor', 'floor', 2, 2, 0)
Rule for SweepLineTask
result: 78 - ('grip1', 'floor', 2, 1, 0)->('grip1', 'floor', 2, 2, 0) = fail
not interpolating
result: 77 - ('floor', 'floor', 2, 1, 0)->('floor', 'floor', 2, 2, 0) = fail
interpolating
sampled one from 62
Rule for AbstractObject
sampled one from 29
sampled one from 64
try: 29 - ('grip1', 'floor', 2, 0, 0)->('floor', 'floor', 2, 0, 0)
result: 2 - ('floor', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 0, 0) = fail
not interpolating
CustomRule call
CustomRule call
sampled one from 4
sampled one from 2
try: 4 - ('floor', 'floor', 2, 0, 0)->('floor', 'grip1', 2, 0, 0)
try: 62 - ('floor', 'floor', 2, 0, 0)->('floor', 'grip1', 2, 0, 0)
('floor', 'grip1', 2, 0, 0)->('floor', 'grip1', 2, 1, 0) , diff: [3]
try: 64 - ('floor', 'grip1', 2, 0, 0)->('floor', 'floor', 2, 0, 0)
CustomRule call
sampled one from 80
Rule for SweepLineTask
try: 2 - ('f

make custom reservation: [<pkg.planning.scene.State instance at 0x7f00290bdb40>, ('grip1', 'floor', 2, 1, 0)]
Rule for AbstractObject
Rule for AbstractObject
CustomRule call
got reserved one from 85
('floor', 'floor', 2, 0, 0)->('floor', 'floor', 2, 1, 0) , diff: [3]
('floor', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 0, 0) , diff: [0]
result: 6 - ('floor', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 0, 0) = fail
result: 73 - ('floor', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 0, 0) = success
result: 34 - ('floor', 'grip1', 2, 0, 0)->('floor', 'floor', 2, 0, 0) = fail
branching: 73->85 (6.94/100.0 s, steps/err: 25(727.918148041 ms)/0.00194738904794)
got reserved one from 85
try: 85 - ('grip1', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 0, 0)
Rule for SweepLineTask
try: 85 - ('grip1', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 1, 0)
sampled one from 29
result: 83 - ('floor', 'floor', 2, 0, 0)->('floor', 'floor', 2, 1, 0) = fail
not interpolating
sampled one from 62
try: 29 - ('grip1', 'floor', 2

Reserve next wp from 90
try: 46 - ('floor', 'floor', 2, 0, 0)->('floor', 'grip1', 2, 0, 0)
constrained motion tried: True
not interpolating
make custom reservation: [('grip1', 'floor', 2, 1, 0)]
not interpolating
Removing 90 from [90] as next wp is reserved
not interpolating
not interpolating
not interpolating
CustomRule call
result: 86 - ('grip1', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 0, 0) = success
make custom reservation: [('floor', 'grip1', 2, 2, 0)]
CustomRule call
CustomRule call
result: 53 - ('floor', 'floor', 2, 1, 0)->('floor', 'floor', 2, 2, 0) = success
CustomRule call
('grip1', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 1, 0) , diff: [3]
('floor', 'floor', 2, 0, 0)->('floor', 'grip1', 2, 0, 0) , diff: [1]
branching: 86->91 (7.34/100.0 s, steps/err: 27(215.981006622 ms)/7.31388304108e-16)
('floor', 'grip1', 2, 0, 0)->('floor', 'floor', 2, 0, 0) , diff: [1]
Rule for AbstractObject
Rule for SweepLineTask
result: 84 - ('floor', 'grip1', 2, 0, 0)->('floor', 'grip1', 2, 1, 0) =

('floor', 'floor', 2, 0, 0)->('floor', 'grip1', 2, 0, 0) , diff: [1]
Rule for AbstractObject
try homing
make custom reservation: [<pkg.planning.scene.State instance at 0x7f00290d8e10>, ('floor', 'grip1', 2, 1, 0)]
result: 2 - ('floor', 'floor', 2, 0, 0)->('floor', 'grip1', 2, 0, 0) = success
branching: 2->97 (7.72/100.0 s, steps/err: 50(1086.07602119 ms)/0.00196792725638)
======================= terminated 2: first answer acquired from other agent ===============================
transition motion tried: False
CustomRule call
('floor', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 0, 0) , diff: [0]
Rule for AbstractObject
result: 68 - ('floor', 'floor', 2, 0, 0)->('grip1', 'floor', 2, 0, 0) = fail
======================= terminated 6: first answer acquired from other agent ===============================
transition motion tried: True
CustomRule call
('floor', 'grip1', 2, 0, 0)->('floor', 'floor', 2, 0, 0) , diff: [1]
Rule for AbstractObject
try homing
make custom reservation: [<pkg.planning.

In [34]:
ppline.play_schedule(snode_schedule, period=0.03)

In [35]:
ppline.wait_procs()
time.sleep(0.5)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.sort_schedule(schedules)
snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

In [36]:
from_state = snode_schedule[-1].state
gtimer.reset()
goal_nodes = [("floor","floor", 2, 2, 2)]
# goal_nodes = [("floor","floor","floor", 2, 2, 2)]
gtimer.tic("plan")
ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
              timeout_loop=100, multiprocess=True, timeout=1, timeout_constrained=5,
              wait_proc=False)
gtimer.toc("plan")

Use 20/20 agents
CustomRule call
make custom reservation: [('floor', 'grip1', 2, 2, 1)]
got reserved one from 0
try: 0 - ('floor', 'grip1', 2, 2, 0)->('floor', 'grip1', 2, 2, 1)
not interpolating
sampled one from 0
try: 0 - ('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0)
not interpolating
sampled one from 0
CustomRule call
try: 0 - ('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0)
not interpolating
transition motion tried: True
('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0) , diff: [1]
Rule for AbstractObject
CustomRule call
('floor', 'grip1', 2, 2, 0)->('floor', 'grip1', 2, 2, 1) , diff: [4]
Rule for SweepLineTask
CustomRule call
result: 0 - ('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0) = fail
Reserve next wp from 1
('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0) , diff: [1]
Removing 1 from [1] as next wp is reserved
sampled one from 0
make custom reservation: [('floor', 'grip1', 2, 2, 2)]
try: 0 - ('floor', 'grip1', 2, 2, 0)->('floor', 'f

194.0438747406006

CustomRule call
interpolating
('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0) , diff: [1]
sampled one from 0
Rule for AbstractObject
result: 0 - ('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0) = fail
try: 0 - ('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0)
sampled one from 0
not interpolating
try: 0 - ('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0)
not interpolating
CustomRule call
('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0) , diff: [1]
Rule for AbstractObject
result: 0 - ('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0) = fail
sampled one from 0
try: 0 - ('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0)
not interpolating
CustomRule call
('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0) , diff: [1]
Rule for AbstractObject
result: 0 - ('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0) = fail
sampled one from 0
try: 0 - ('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0)
not interpolating
CustomRule call
('f

('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0) , diff: [1]
Rule for AbstractObject
CustomRule call
CustomRule call
('floor', 'floor', 2, 2, 0)->('floor', 'grip1', 2, 2, 0) , diff: [1]
Rule for AbstractObject
result: 0 - ('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0) = fail
Rule for AbstractObject
('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0) , diff: [1]
sampled one from 2
try: 2 - ('floor', 'floor', 2, 2, 0)->('grip1', 'floor', 2, 2, 0)
result: 0 - ('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0) = fail
CustomRule call
not interpolating
try homing
Rule for AbstractObject
('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0) , diff: [1]
try: 2 - ('floor', 'floor', 2, 2, 0)->('grip1', 'floor', 2, 2, 0)
not interpolating
Rule for AbstractObject
Rule for AbstractObject
CustomRule call
result: 0 - ('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0) = fail
CustomRule call
result: 0 - ('floor', 'grip1', 2, 2, 0)->('floor', 'floor', 2, 2, 0) = fai

Rule for AbstractObject
result: 2 - ('floor', 'floor', 2, 2, 0)->('grip1', 'floor', 2, 2, 0) = fail
result: 2 - ('floor', 'floor', 2, 2, 0)->('grip1', 'floor', 2, 2, 0) = fail
CustomRule call
result: 2 - ('floor', 'floor', 2, 2, 0)->('floor', 'grip1', 2, 2, 0) = fail
('floor', 'floor', 2, 2, 0)->('grip1', 'floor', 2, 2, 0) , diff: [0]
try: 2 - ('floor', 'floor', 2, 2, 0)->('floor', 'grip1', 2, 2, 0)
sampled one from 2
sampled one from 2
Rule for AbstractObject
sampled one from 2
try: 2 - ('floor', 'floor', 2, 2, 0)->('grip1', 'floor', 2, 2, 0)
try: 2 - ('floor', 'floor', 2, 2, 0)->('floor', 'grip1', 2, 2, 0)
not interpolating
try: 2 - ('floor', 'floor', 2, 2, 0)->('floor', 'grip1', 2, 2, 0)
not interpolating
joint motion tried: True
result: 2 - ('floor', 'floor', 2, 2, 0)->('grip1', 'floor', 2, 2, 0) = fail
not interpolating
not interpolating
CustomRule call
sampled one from 2
CustomRule call
('floor', 'floor', 2, 2, 0)->('floor', 'grip1', 2, 2, 0) , diff: [1]
try: 2 - ('floor', 'floor

...

try: 58 - ('floor', 'grip1', 2, 2, 1)->('floor', 'grip1', 2, 2, 2)
result: 51 - ('floor', 'grip1', 2, 2, 0)->('floor', 'grip1', 2, 2, 0) = success
make custom reservation: [<pkg.planning.scene.State instance at 0x7f00290c2230>, ('floor', 'floor', 2, 2, 1)]
transition motion tried: True
CustomRule call
Rule for AbstractObject
('floor', 'floor', 2, 2, 0)->('grip1', 'floor', 2, 2, 0) , diff: [0]
result: 2 - ('floor', 'floor', 2, 2, 0)->('grip1', 'floor', 2, 2, 0) = fail
result: 23 - ('grip1', 'floor', 2, 2, 0)->('floor', 'floor', 2, 2, 0) = success
branching: 51->60 (4.88/100.0 s, steps/err: 34(398.495912552 ms)/3.6712384296e-16)
interpolating
branching: 23->59 (4.89/100.0 s, steps/err: 28(471.888780594 ms)/0.000779707689283)
transition motion tried: True
joint motion tried: True
Rule for AbstractObject
got reserved one from 60
CustomRule call
CustomRule call
got reserved one from 59
('floor', 'grip1', 2, 2, 0)->('floor', 'grip1', 2, 2, 1) , diff: [4]
('grip1', 'floor', 2, 2, 0)->('grip1'

Removing 70 from [70] as next wp is reserved
Rule for SweepLineTask
branching: 2->69 (5.3/100.0 s, steps/err: 114(4296.71311378 ms)/0.00182359723505)
make custom reservation: [('floor', 'grip1', 2, 2, 2)]
interpolating
result: 64 - ('floor', 'floor', 2, 2, 0)->('floor', 'floor', 2, 2, 1) = fail
got reserved one from 69
result: 65 - ('floor', 'grip1', 2, 2, 0)->('floor', 'grip1', 2, 2, 1) = success
got reserved one from 70
try: 70 - ('floor', 'grip1', 2, 2, 1)->('floor', 'grip1', 2, 2, 2)
CustomRule call
try: 69 - ('floor', 'grip1', 2, 2, 0)->('floor', 'grip1', 2, 2, 1)
branching: 65->70 (5.34/100.0 s, steps/err: 8(113.50107193 ms)/0.00193208397278)
interpolating
constrained motion tried: False
sampled one from 5
not interpolating
CustomRule call
try: 5 - ('floor', 'floor', 2, 2, 0)->('grip1', 'floor', 2, 2, 0)
('grip1', 'floor', 2, 2, 1)->('grip1', 'floor', 2, 2, 2) , diff: [4]
not interpolating
('floor', 'grip1', 2, 2, 1)->('floor', 'grip1', 2, 2, 2) , diff: [4]
Rule for SweepLineTask

got reserved one from 77
CustomRule call
result: 29 - ('floor', 'grip1', 2, 2, 1)->('floor', 'grip1', 2, 2, 2) = success
('floor', 'floor', 2, 2, 0)->('floor', 'floor', 2, 2, 0) , diff: []
try: 77 - ('floor', 'grip1', 2, 2, 0)->('floor', 'grip1', 2, 2, 1)
got reserved one from 78
branching: 29->76 (6.22/100.0 s, steps/err: 57(3352.69093513 ms)/0.00121065611042)
not interpolating
result: 72 - ('floor', 'grip1', 2, 2, 2)->('floor', 'floor', 2, 2, 2) = success
result: 64 - ('floor', 'floor', 2, 2, 0)->('floor', 'floor', 2, 2, 0) = fail
try: 78 - ('floor', 'grip1', 2, 2, 0)->('floor', 'grip1', 2, 2, 1)
got reserved one from 76
not interpolating
sampled one from 79
try: 76 - ('floor', 'grip1', 2, 2, 2)->('floor', 'grip1', 2, 2, 2)
branching: 72->79 (6.24/100.0 s, steps/err: 25(403.317928314 ms)/0.00165158976945)
try: 79 - ('floor', 'floor', 2, 2, 2)->('floor', 'grip1', 2, 2, 2)
not interpolating
transition motion tried: True
======================= terminated 15: first answer acquired =====

======================= terminated 0: first answer acquired from other agent ===============================
constrained motion tried: True
CustomRule call
('floor', 'grip1', 2, 2, 1)->('floor', 'grip1', 2, 2, 2) , diff: [4]
Rule for SweepLineTask
Check home: 2 in [0, 2]
reserve homing = 85
make custom reservation: [<pkg.planning.scene.State instance at 0x7f00290c23c0>]
result: 66 - ('floor', 'grip1', 2, 2, 1)->('floor', 'grip1', 2, 2, 2) = success
branching: 66->85 (8.27/100.0 s, steps/err: 57(3020.14398575 ms)/0.00136772060092)
======================= terminated 12: first answer acquired from other agent ===============================
constrained motion tried: False
CustomRule call
('floor', 'grip1', 2, 2, 1)->('floor', 'grip1', 2, 2, 2) , diff: [4]
Rule for SweepLineTask
result: 34 - ('floor', 'grip1', 2, 2, 1)->('floor', 'grip1', 2, 2, 2) = fail
======================= terminated 17: first answer acquired from other agent ===============================
constrained motion tried: F

Process Process-61:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "pkg/planning/pipeline.py", line 153, in __search_loop
    display=display, dt_vis=dt_vis, **kwargs)
  File "pkg/planning/pipeline.py", line 200, in test_connection
    self.mplan.plan_transition(from_state, to_state, redundancy_dict=redundancy_dict, **kwargs)
  File "pkg/planning/motion/interface.py", line 105, in plan_transition
    redundancy_values=redundancy_values, **kwargs)
  File "pkg/planning/motion/moveit/moveit_planner.py", line 171, in plan_algorithm
    group_name, tuple(to_Q), tuple(from_Q), timeout=timeout_joint)
  File "pkg/planning/motion/moveit/moveit_py.py", line 123, in plan_joint_motion_py
    JointState(self.joint_num, *Q_init), plannerconfig, timeout)
KeyboardInterrupt


In [37]:
ppline.play_schedule(snode_schedule, period=0.03)

In [38]:
ppline.wait_procs()
time.sleep(0.5)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.sort_schedule(schedules)
snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

In [39]:
snode_schedule = ppline.add_return_motion(snode_schedule, initial_state, timeout=5)
ppline.play_schedule(snode_schedule, period=0.03)

joint motion tried: True
joint motion tried: True


## MUST CHECK! no more than 1 trial from non-terminal nodes
## MUST CHECK! no more than 1 trial to same framed motion
## MUST CHECK! efficient binding/param sampling for gripper

In [41]:
for k, v in tplan.custom_rule.call_count_dict.items():
    print("{}: {} - {}".format(k,tplan.snode_dict[k].state.node, sorted(v)))

0: ('floor', 'grip1', 2, 2, 0) - [('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 0), ('floor', 'grip1', 2, 2, 1)]
1: ('floor', 'grip1', 2, 2, 1) - [('floor', 'grip1', 2, 2, 2)]
2: ('floor', 'floor', 2, 2, 0) - [('floor', 'floor', 2, 2, 0), ('floor', 'floor', 2, 2, 1), ('floor', 'grip1', 2, 2, 0), ('floor', 'grip1', 2, 2, 0), ('floor', 'grip1', 2, 2, 0), ('floor', 'grip1', 2, 2, 0), ('floor', 'grip1', 2, 2, 0),

## play searched plan

In [ ]:
ppline.play_schedule(snode_schedule[:2], period=0.001)
pscene.set_object_state(initial_state)
gscene.show_pose(crob.home_pose)
time.sleep(0.5)
gscene.show_pose(crob.home_pose)

In [ ]:
snode_schedule = ppline.add_return_motion(snode_schedule)

In [ ]:
ppline.play_schedule(snode_schedule, period=0.03)

In [ ]:
print("schedule length: {}".format(len(snode_schedule)))
for i_s,  snode in enumerate(snode_schedule):
    print("{}: {}".format(i_s, snode.state.node))

## NOTE
* 위에 가운데 열 어떻게  0,1,1,1,1,2가 나오지? -> 일방통행 작업 플래그 추가
* 균등 샘플링 - 샘플 할때마다 노드별/전환별 확률 조정
* goal-directed extension 추가.

In [ ]:
tplan.node_dict[('grip0', 'goal', 0, 1, 2)]

## extend preserving goal-matching items

In [ ]:
print(gtimer)

## extend only no reservation

In [ ]:
print(gtimer)

## no extension

In [ ]:
print(gtimer)

## extend_toward goal

In [ ]:
print(gtimer)